In [2]:
!ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
!ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 74aebb552ea7... 100% ▕████████████████▏  68 MB                         
pulling ef1438627c47... 100% ▕████████████████▏  190 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 6f85a640a97c... 100% ▕

In [3]:
pip install ollama

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sezer\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
dataset = []
with open('feedback_data.json', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')

Loaded 47 entries


In [5]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))

In [6]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')


Added chunk 1/47 to the database
Added chunk 2/47 to the database
Added chunk 3/47 to the database
Added chunk 4/47 to the database
Added chunk 5/47 to the database
Added chunk 6/47 to the database
Added chunk 7/47 to the database
Added chunk 8/47 to the database
Added chunk 9/47 to the database
Added chunk 10/47 to the database
Added chunk 11/47 to the database
Added chunk 12/47 to the database
Added chunk 13/47 to the database
Added chunk 14/47 to the database
Added chunk 15/47 to the database
Added chunk 16/47 to the database
Added chunk 17/47 to the database
Added chunk 18/47 to the database
Added chunk 19/47 to the database
Added chunk 20/47 to the database
Added chunk 21/47 to the database
Added chunk 22/47 to the database
Added chunk 23/47 to the database
Added chunk 24/47 to the database
Added chunk 25/47 to the database
Added chunk 26/47 to the database
Added chunk 27/47 to the database
Added chunk 28/47 to the database
Added chunk 29/47 to the database
Added chunk 30/47 to th

In [7]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [8]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]

In [ ]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

formatted_chunks = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])
result = f"{formatted_chunks}"

instruction_prompt = f"""You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{formatted_chunks}
"""

In [ ]:
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


NameError: name 'ollama' is not defined